### TODO
- Factor out classes into .py-files to make notebook more concise
- Find package versions where we don't need to filter warnings (older packages)
- Set up pseudocode for the tournament;
    - players, saving models, etc.

In [70]:
# ignore deprecation warnings ('safe' as long as we don't update packages)
from warnings import filterwarnings
filterwarnings("ignore")
from warnings import filterwarnings
filterwarnings("ignore")
from copy import deepcopy
import numpy as np

from joblib import Parallel, delayed
import gym
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
from random import choice
import sys

from src.CNN import CNN
from src.MCTS import Monte_Carlo_Tree_Search

In [71]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BOARD_SIZE = 5

# Convolutional Neural Network

In [72]:
# def train_model(model, lr, momentum):
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
#     model.to(device)
    
#     for epoch in range(10):
#         mcts = Monte_Carlo_Tree_Search(BOARD_SIZE, model) # new tree
#         mcts.run_game() # run a single game to completion
#         x, y = mcts.get_tree_data() # get data
#         running_loss = .0 
#         for i in range(len(x)):
#             inputs, labels = torch.tensor(x[i], device=device), torch.tensor(y[i], device=device)
#             labels = F.softmax(labels)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()

#             if i % 10 == 9:
#                 print(f'[{epoch:3d}, {i + 1:3d}] loss: {running_loss / 10:.5f}')
#                 running_loss = 0.0

# def train(n_games=1000):

#     model = CNN()
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

#     all_x, all_y = [], []

#     # Simulate N games
#     for _ in range(n_games):
#         mcts = Monte_Carlo_Tree_Search(BOARD_SIZE, model)
#         mcts.run(10_000)
#         x, y = mcts.get_tree_data()

#         for i in range(len(x)):
#             inputs, labels = torch.tensor(x[i], device=device), torch.tensor(y[i], device=device)
#             labels = F.softmax(labels)
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
            
#             all_x.append(x[i])
#             all_y.append(y[i])
        
#         # After training, check accuracy against previous data
#         acc_values.append(model.avg_accuracy(all_x, all_y).item())
#         loss_values.append(game_loss/len(x))

#     print(f'''Finished running {n_games} games
#     Time..............: {total_time}
#     Avg. time per game: {total_time/n_games}''')
#     return model, loss_values, acc_values


In [73]:
# model, loss_values, acc_values = train(1000)

# MCTS and CNN combined

In [74]:
# def generate_game(x, y, model):
#     mcts = Monte_Carlo_Tree_Search(BOARD_SIZE, model)
#     # mcts.run_game()
#     mcts.run(1000)
#     a, b = mcts.get_tree_data()
#     x.append(a)
#     y.append(b)

# def generate_games(x, y, n_games, model):
#     print(f"Generating {n_games} games")
#     Parallel(n_jobs=10)(delayed(generate_game)(x, y, model) for _ in range(1, n_games))

# def train_model(model, criterion, optimizer, x, y):
#     print("Training model")
#     for i in range(len(x)):
#         inputs, labels = torch.tensor(x[i], device=device), torch.tensor(y[i], device=device)
#         labels = F.softmax(labels)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

In [75]:
# x, y = [], []
# generate_games(x, y, 1000, None)

# model = CNN()
# model.to(device=device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# train_model(model, criterion, optimizer, x, y)

# print("Exporting model")
# torch.save(model.state_dict(), f"models/only-mcts.pth")

In [76]:
# # model = CNN()
# # model.load_state_dict(torch.load("models/only-mcts.pth"))
# # model.eval()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# for i in range(2, 6):
#     # x, y = [], []
#     generate_games(x, y, 1000, model)
#     train_model(model, criterion, optimizer, x, y)
#     torch.save(model.state_dict(), f"models/{i}-times.pth")

In [77]:
players = []
for i in range(5):
    player = CNN()
    player.to(device)
    player.load_state_dict(torch.load(f"models/{i}-times.pth"))
    players.append(player)
print(players)

[CNN(
  (conv1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=26, bias=True)
), CNN(
  (conv1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=26, bias=True)
), CNN(
  (conv1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=26, bias=True)
), CNN(
  (conv1): Conv2d(6, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1

In [78]:
def play_model_vs_model_no_render(model1 : CNN, model2 : CNN, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        weights = model1.forward(go_env.state()).cpu().detach().numpy()
        for i in range(len(weights) - 1):
            action2d = i // BOARD_SIZE, i % BOARD_SIZE, i
            if go_env.state()[3, action2d[0], action2d[1]] == 1:
                weights[i] = 0.0
        _, _, done, _ = go_env.step(weights.argmax())

        if done: continue

        weights = model2.forward(go_env.state()).cpu().detach().numpy()
        for i in range(len(weights) - 1):
            action2d = i // BOARD_SIZE, i % BOARD_SIZE, i
            if go_env.state()[3, action2d[0], action2d[1]] == 1:
                weights[i] = 0.0
        _, _, done, _ = go_env.step(weights.argmax())
        turn_nr += 1
        if turn_nr > 300: break
    return go_env

In [79]:
def play_tournament(wins, draws, games_played, b_w):
    # print("Starting a tournament")
    # start1 = time.time()
    for player1_index in range(len(players)):
        for player2_index in range(len(players)):
            if player1_index == player2_index: continue
            games_played[player1_index] += 1
            games_played[player2_index] += 1
            p1, p2 = players[player1_index], players[player2_index]
            # print(f"  Starting a game between {player1_index} and {player2_index}", end="")
            # start = time.time()
            go_env = play_model_vs_model_no_render(p1, p2, gym.make('gym_go:go-v0', size=BOARD_SIZE, komi=0, reward_method='heuristic'))
            # stop = time.time()
            if go_env.reward() > 0:
                wins[player1_index] += 1
                b_w[0] += 1
            elif go_env.reward() < 0:
                wins[player2_index] += 1
                b_w[1] += 1
            else:
                draws[player1_index] += 1
                draws[player2_index] += 1
            # print(f", reward: {go_env.reward()}, time: {stop-start} sec")
    # stop1 = time.time()
    # print(f"Tournament took {stop1-start1}")
b_w = [0]*2
wins, draws, games_played = [0]*len(players), [0]*len(players), [0]*len(players)
# Parallel(n_jobs=10)(delayed(play_tournament)(wins, draws, games_played, b_w) for _ in range(1000))
for game_number in range(1000):
    play_tournament(wins, draws, games_played, b_w)

TypeError: play_tournament() missing 1 required positional argument: 'b_w'

In [ ]:
print(b_w)
print(wins)
print(games_played)
print(draws)

[0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
